## Network attribute Validation Checks


#### Import Libraries 

In [ ]:
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
# from nxviz import CircosPlot
from shapely.geometry import LineString

### Load Network data

#### Create Function: *segment_heading*: 
Calculate Heading for both Initial segment and Final segment of Link

In [ ]:
def segment_heading(line, first_segment=True, flip=False):
    if flip:
        line = LineString(line.coords[::-1])
    
    segment = line.coords[:2] if first_segment else line.coords[-2:]
    
    from_pt = segment[0]
    to_pt = segment[1]
    
    y_delta = to_pt[1] - from_pt[1]
    x_delta = to_pt[0] - from_pt[0]

    angle_radians = np.arctan2(y_delta, x_delta)
    if angle_radians < 0:
        angle_radians = 2 * np.pi + angle_radians;

    # Deal with the angles being from East Counterclockwise to Northerly bearings
    degrees = 90 - angle_radians * 180 / np.pi
    if degrees < 0:
        degrees = degrees + 360

    return degrees

#### Load Node and Line Network data from Shapefiles
Use GeoPandas to load Geospatial data  
Call function 'segment_heading' to calculate Heading

In [ ]:
line_shp = gpd.read_file(os.path.join('..', 'data', 'Chatt_Master.shp')).to_crs(epsg=2274)
node_shp = gpd.read_file(os.path.join('..', 'data', 'Chatt_Master_Node.shp')).to_crs(epsg=2274)

# Heading AB
line_shp['AB_START_HEADING'] = line_shp['geometry'].apply(segment_heading, first_segment=True)
line_shp['AB_END_HEADING'] = line_shp['geometry'].apply(segment_heading, first_segment=False)
# Heading BA
line_shp['BA_START_HEADING'] = line_shp['geometry'].apply(segment_heading, first_segment=True, flip=True)
line_shp['BA_END_HEADING'] = line_shp['geometry'].apply(segment_heading, first_segment=False, flip=True)

### Build Network Graph

#### Turn Two-Way Links into 2 One-Way Links for Directional Flow.
#### Create Function: *directional_links*
Keep AB Links and produce BA Links, carrying over link attributes based on the direction of flow.

In [ ]:
def directional_links(atts, line_shp):
    dirs = {'keep':'AB', 'invert':'BA'}
    dfs = {}
    
    for flow,d in dirs.items():
        links = line_shp[line_shp['{}_LANES'.format(d)] > 0].copy()
        rename_dict = {'{}{}'.format(d, a):a for a in atts}
        links = links.rename(columns=rename_dict)
        
        if flow == 'invert':
            links['A'] = links['TO_ID']
            links['B'] = links['FROM_ID']
            links = links.drop(columns=['FROM_ID', 'TO_ID']).rename(columns={'A': 'FROM_ID', 'B': 'TO_ID'})
        
        dfs['{}_links'.format(d)] = links
    
    links = pd.concat(dfs.values(), ignore_index=True)
    
    drop_dict = ['{}{}'.format(d, a) for a in atts for d in dirs.values()]
    links = links.drop(columns=drop_dict)
    
    return links

#### Generate Directional links with Attributes
####  Call Function: *directional_links*
Use list of Directional Attributes to carry over, e.g. 'AB_LANES' and 'BA_LANES', to '_LANES' 

In [ ]:
# List of Directional Attributes
atts = [
    '_LINKDIR',
    '_LINKDTR',
    '_LANES',
    '_PARKING',
    '_TRIMS',
    '_AADT',
    '_CAR_ADT',
    '_SUT_ADT',
    '_MUT_ADT',
    '_BASEVOL',
    '_AFFTIME',
    '_AFFSPD',
    '_UCDELAY',
    '_DLYCAP',
    '_AMCAP',
    '_PMCAP',
    '_BPRA',
    '_BPRB',
    '_START_HEADING',
    '_END_HEADING'
]
# Generate Directional Links
line_shp = directional_links(atts, line_shp)

####  Build the Directional Graph: *g*
#### Call Method: *nx.from_pandas_edgelist*

In [ ]:
g = nx.from_pandas_edgelist(
    line_shp,
    source='FROM_ID',
    target='TO_ID',
    edge_attr=True,
    create_using=nx.DiGraph()
)

assert nx.is_directed(g)

## Run Node VALIDATION TESTS

### Summarize IN / OUT Base Volumes

In [ ]:
node_feed = []

for node in node_shp['ID']:
    if node not in g:
        print('Node not in network: {}'.format(node))
        continue

    values = [node,
              sum([d['_BASEVOL'] for a, b, d in g.in_edges(node, data=True) if not np.isnan(d['_BASEVOL'])]),
              sum([d['_BASEVOL'] for a, b, d in g.out_edges(node, data=True) if not np.isnan(d['_BASEVOL'])]),
             ]
    node_feed.append(values)
    
cols = ['ID', 'in_BASEVOL', 'out_BASEVOL']

nodes = pd.DataFrame(node_feed, columns=cols).fillna(0)


nodes['_BASEVOL_CHG'] = (((nodes['out_BASEVOL'] - nodes['in_BASEVOL']) / nodes['in_BASEVOL'])*100).round(2)
# nodes['_BASEVOL_CHG'] = (nodes['out_BASEVOL'] - nodes['in_BASEVOL']) / nodes['in_BASEVOL']
nodes['_BASEVOL_CHG'] = nodes['_BASEVOL_CHG'].replace([np.inf, -np.inf], np.nan).fillna(0)


node_BASEVOL = pd.merge(node_shp[['ID', 'CENTROID', 'geometry']], nodes, on='ID', how='left')
   
node_BASEVOL.head()

In [ ]:
shp_export = os.path.join('..', 'data', 'Node_BASEVOL.shp')
node_BASEVOL.to_file(driver = 'ESRI Shapefile', filename=shp_export)

In [ ]:
plt.scatter(
    node_BASEVOL['in_BASEVOL'], 
    node_BASEVOL['out_BASEVOL'], 
    c=node_BASEVOL['CENTROID'].fillna(0), 
    cmap=plt.cm.coolwarm
    )
plt.colorbar()

In [ ]:
node_BASEVOL_NonZero = node_BASEVOL[node_BASEVOL['_BASEVOL_CHG']!=0]
plt.scatter(
    node_BASEVOL_NonZero['in_BASEVOL'], 
    node_BASEVOL_NonZero['out_BASEVOL'], 
    c=node_BASEVOL_NonZero['CENTROID'].fillna(0), 
    cmap=plt.cm.coolwarm
    )
plt.colorbar()

In [ ]:
node_BASEVOL.plot(column='_BASEVOL_CHG',
                  markersize=node_BASEVOL['_BASEVOL_CHG'],
#                   legend=True,
                  figsize=(8, 12)
                 )

In [ ]:
### CHECK
node = 14955

[print(d['_BASEVOL']) for a, b, d in g.in_edges(node, data=True) if not np.isnan(d['_BASEVOL'])]
[print(d['_BASEVOL']) for a, b, d in g.out_edges(node, data=True) if not np.isnan(d['_BASEVOL'])]

In [ ]:
### CHECK
node = 896

[print(a, b, d['ID']) for a, b, d in g.in_edges(node, data=True)]
[print(a, b, d['ID']) for a, b, d in g.out_edges(node, data=True)]

In [ ]:
line_shp[['FROM_ID', 'TO_ID', 'ID']][line_shp['ID'].isin([9205, 37252])]

In [ ]:
print(g.edges[10415, 896]['ID'])
print(g.edges[896, 10415]['ID'])

In [ ]:
list(node_shp)